# Grafos de fluxos

Na sua versão mais simples, um *grafo de fluxos* é um grafo orientado onde cada aresta modela uma "conduta" onde pode fluir "alguma coisa" (gás, água, trânsito automóvel, etc) sujeita a uma capacidade máxima. Mais em informação em https://en.wikipedia.org/wiki/Flow_network.

Usando o `networkx` podemos modelar um grafo de fluxos usando um `DiGraph` (*directed graph*), usando atributos adicionais para representar a capacidade máxima de cada arco e o respectivo fluxo efectivo. Por exemplo, o seguinte código cria um grafo de fluxos com 6 nodos e 10 arcos.

In [ ]:
import networkx as nx

graph = nx.DiGraph()
graph.add_edge(0,1,capacity=16)
graph.add_edge(0,2,capacity=13)
graph.add_edge(1,2,capacity=10)
graph.add_edge(1,3,capacity=12)
graph.add_edge(2,1,capacity=4)
graph.add_edge(2,4,capacity=14)
graph.add_edge(3,2,capacity=9)
graph.add_edge(3,5,capacity=20)
graph.add_edge(4,3,capacity=7)
graph.add_edge(4,5,capacity=4)

Podemos desenhar este grafo de fluxos usando a seguinte função que assume que o grafo é planar e que coloca como fluxo efectivo o caracter `?` caso este atributo não esteja definido.

In [ ]:
def draw_flow_network(graph):
    pos = nx.planar_layout(graph)
    labels = {}
    for (o,d) in graph.edges():
        if 'flow' in graph[o][d]:
            labels[(o,d)] = str(graph[o][d]['flow']) + ' / ' + str(graph[o][d]['capacity'])
        else:
            labels[(o,d)] = '? / ' + str(graph[o][d]['capacity'])                        
    nx.draw_networkx(graph,pos)
    nx.draw_networkx_edge_labels(graph,pos,labels,label_pos=0.3)


draw_flow_network(graph)

## Max-flow

Um dos problemas clássicos com grafos de fluxos consiste em determinar qual o fluxo máximo que pode existir entre dois nodos concretos do grafo.

Usando programação inteira mista (com variáveis inteiras e reais) podemos determinar este fluxo máximo da seguinte forma.

### Análise do problema

Dado um grafo orientado $(V,E)$, uma capacidade $C_e \ge 0$ para cada arco $e \in E$, um nodo $s$ correspondente à origem e um nodo $t$ correspondente ao destino, precisamos de uma variável $f_e$ do tipo real por cada arco $e \in E$, que irá representar o fluxo efectivo nesse arco. Para encontrar o fluxo máximo entre $s$ e $t$ basta maximizar

$$
\sum_{(s,v) \in E} f_{(s,v)} - \sum_{(v,s) \in E} f_{(v,s)}
$$

ou seja, a soma do fluxos efectivos com origem em $s$ (alternativamente, poderia-se maximizar a soma dos fluxos efectivos com destino $t$), sujeito às seguintes restrições:
- A capacidade de cada arco é respeitada
- O fluxo que sai de cada nodo é igual ao que entra (exceptuando em $s$ e $t$)

### Exercício 1

Formalize estas restrições.

### Exercício 2

Implemente a função `max_flow` que dado um grafo de fluxos, determina o fluxo máximo entre `s` e `t`, definindo também o atributo `flow` com o fluxo efectivo de cada arco necessário para atingir esse fluxo máximo. Em `networkx` pode usar o método `edges` para aceder aos arcos de um grafo, e os métodos `in_edges` e `out_edges` para aceder aos arcos de entrada e saída de um determinado vértice.

In [ ]:
from ortools.linear_solver import pywraplp

def max_flow(graph,s,t):
    solver = pywraplp.Solver.CreateSolver('SCIP')

    f = {}
    for (u,v) in graph.edges():
        cap = graph[u][v]['capacity']
        f[(u,v)] = solver.NumVar(0,cap,f'f_{u}_{v}')
    
 
    # fluxo entrada = fluxo saida
    
    for node in graph.nodes():
        if node in [s,t]:
            continue
        in_node = solver.Sum(f[(u,node)] for (u,node) in graph.in_edges(node))
        out_node = solver.Sum(f[(node,v)]for (node,v) in graph.out_edges(node))
        solver.Add(in_node == out_node)

    out_s = solver.Sum(f[s,v] for (s,v) in graph.out_edges(s))
    in_s = solver.Sum(f[u,s] for (u,s) in graph.in_edges(s))

    solver.Maximize(out_s - in_s)

    status = solver.Solve()

    if status != pywraplp.Solver.OPTIMAL:
        print("Não encontrou solução otima")

    for (u,v) in graph.edges():
        graph[u][v]['flow'] = f[(u,v)].solution_value()
    

    return solver.Objective().Value()
    
assert max_flow(graph,0,5) == 23
draw_flow_network(graph)

## Min-cut

Um dos resultados fundamentais sobre grafos de fluxos, conforme https://en.wikipedia.org/wiki/Max-flow_min-cut_theorem, diz que o fluxo máximo entre dois vértices é igual à soma da capacidade dos arcos que estabelecem um *corte mínimo* entre esses vértices. Num grafo um *corte mínimo* é um conjunto de arcos que divide os nodos em dois conjuntos disjuntos satisfazendo algum critério de minimalidade. Neste caso estamos interessados em dividir o grafo em dois conjuntos $S$ e $T$ onde a origem $s \in S$ e o destino $t \in T$, minimizando a capacidade total dos arcos que pertecem ao corte, ou seja, a capacidade total dos arcos que ligam $S$ a $T$.

### Análise do problema

Dado um grafo orientado $(V,E)$, uma capacidade $C_e \ge 0$ para cada arco $e \in E$, um nodo $s$ correspondente à origem e um nodo $t$ correspondente ao destino, precisamos de uma variável binária $d_{(a,b)}$ por cada aresta $(a,b) \in E$, que terá o valor 1 se a respectiva aresta pertencer ao corte entre $S$ e $T$ (ou seja, se $a \in S$ e $b \in T$), e de uma variável binária $x_v$ por cada vértice $v \in V$, que terá o valor 1 se o respectivo vértice pertencer a $S$ (se o valor for 0 é porque $v \in T$). Para encontrar o corte mínimo basta minimizar

$$
\sum_{e \in E} d_{e} \cdot C_e
$$

sujeito às seguintes restrições:
- O nodo $s$ pertence a $S$ e o nodo $t$ pertence a $T$
- Uma aresta com origem em $S$ e destino em $T$ pertence ao corte

### Exercício 3

Formalize estas restrições.

### Exercício 4

Implemente a função `min_cut` que dado um grafo de fluxos determina o fluxo máximo entre `s` e `t`.

In [ ]:
def min_cut(graph,s,t):
    solver = pywraplp.Solver.CreateSolver('SCIP')

    x = {v : solver.BoolVar(f'x_{v}') for v in graph.nodes()}
    
    d = {}
    
    for (a,b) in graph.edges():
        d[(a,b)] = solver.BoolVar(f'd_{a}_{b}')
    
    
    solver.Add(x[s] == 1)
    solver.Add(x[t] == 0)
    for (a,b) in graph.edges():
        solver.Add(d[a,b] >= x[a] - x[b])
    
    obj = solver.Sum(graph[a][b]['capacity'] * d[(a,b)] for (a,b) in graph.edges())
    solver.Minimize(obj)
    
    status = solver.Solve()
    if status != pywraplp.Solver.OPTIMAL:
        print("Não encontrou solução otima")

    return solver.Objective().Value()
    
assert min_cut(graph,0,5) == 23

## Outros problemas de corte mínimo

Outro problema relacionado com o corte mínimo consiste em determinar, para um grafo orientado e dois nodos $s$ e $t$ desse grafo, o número mínimo de arcos que é necessário retirar para que $t$ deixe de ser acessível a partir de $s$.

Uma forma directa de resolver este problema consiste em começar por determinar todos os caminhos (sem ciclos) entre $s$ e $t$ e depois determinar o conjunto mínimo de arestas que intersecta todos esses caminhos.

### Exercício 5

Formalize este problema usando programação inteira. Assuma que dado um grafo $(V,E)$ e os nodos $s$ e $t$, $P$ é o conjunto de todos os caminhos (sem ciclos) entre $s$ e $t$, onde cada caminho é representado pelo conjunto dos arcos que lhe pertencem.

### Exercício 6

Implemente a função `disconnect_all_paths` que dado um grafo `graph` e os nodos `s` e `t` devolve a lista de arestas a remover para que `t` deixe de ser acessível a partir de `s`. Pode usar a função `all_simple_paths` do `networkx` para determinar todos os caminhos sem ciclos entre dois vértices.

In [ ]:
def disconnect_all_paths(graph,s,t):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    d = {}
    for (a,b) in graph.edges():
        d[(a,b)] = solver.BoolVar(f'd_{a}_{b}')
        
    all_paths = list(nx.all_simple_paths(graph,s,t))
    for path in all_paths:
        edges_in_path = list(zip(path[:-1],path[1:]))
        solver.Add(solver.Sum(d[e] for e in edges_in_path) >= 1)
    
    obj = solver.Sum(graph[a][b]['capacity'] * d[(a,b)] for (a,b) in graph.edges())
    solver.Minimize(obj)
    
    status = solver.Solve()
    if status != pywraplp.Solver.OPTIMAL:
        print("Não encontrou solução otima")
        
    removed = [(a,b) for (a,b) in graph.edges() if d[(a,b)].solution_value() == 1]
    
    return removed    
disconnect_all_paths(graph,0,5)

### Exercício 7

Também é possível reduzir este problema ao problema do corte mínimo em grafos de fluxos assumindo simplesmente que a capacidade  dos arcos é 1.

Implemente a função `disconnect_min_cut` que dado um grafo `graph` e os nodos `s` e `t` devolve a lista de arestas a remover para que `t` deixe de ser acessível a partir de `s` usando esta técnica.

In [ ]:
def disconnect_min_cut(graph,s,t):
    solver = pywraplp.Solver.CreateSolver('SCIP')

    x = {v: solver.BoolVar(f'x_{v}') for v in graph.nodes()}

    d = {(a,b): solver.BoolVar(f'd_{a}_{b}') for (a,b) in graph.edges()}

    solver.Add(x[s] == 1)
    solver.Add(x[t] == 0)
    for (a,b) in graph.edges():
        solver.Add(d[(a,b)] >= x[a] - x[b])
    
    obj = solver.Sum(d[(a,b)] for (a,b) in graph.edges())
    
    solver.Minimize(obj)
    
    
    status = solver.Solve()
    if status != pywraplp.Solver.OPTIMAL:
        print("Não encontrou solução otima")
        
    removed = [(a,b) for (a,b) in graph.edges() if d[(a,b)].solution_value() == 1]

    return removed 
disconnect_min_cut(graph,0,5)